<a href="https://colab.research.google.com/github/DayenaJeong/capstone2_SEDA/blob/main/convert_to_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.5 MB/s eta 0:00:00


In [6]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import pickle

In [7]:
def calculate_mean_keypoints_from_file(keypoints_data):

    # 모든 동영상에 대한 평균 키포인트 계산
    mean_keypoints_all_videos = []
    for keypoints_list in keypoints_data['keypoints']:
        # 각 동영상에 대한 키포인트 리스트에서 평균 계산
        mean_keypoints = [[sum(pos) / len(keypoints_list) for pos in zip(*frame)] for frame in zip(*keypoints_list)]
        mean_keypoints_all_videos.append(mean_keypoints)

    return mean_keypoints_all_videos, keypoints_data['labels']

In [8]:
def calculate_keypoint_changes(keypoints_data):
    # 변경된 부분: 이미 로드된 키포인트 데이터를 직접 사용
    # 키포인트 데이터는 각 동영상의 프레임별 키포인트 리스트를 포함하는 리스트

    changes_list = []  # 변화량을 저장할 리스트 초기화

    for keypoints_list in keypoints_data['keypoints']:
        changes = []  # 개별 동영상의 키포인트 변화량을 저장할 리스트
        prev_keypoints = None

        for keypoints in keypoints_list:
            keypoints = np.array(keypoints)
            if prev_keypoints is not None:
                # 현재 프레임과 이전 프레임의 키포인트 사이의 변화량 계산
                change = np.abs(keypoints - prev_keypoints)
                changes.append(change)
            prev_keypoints = keypoints

        # 모든 변화량의 평균 계산
        if changes:
            mean_changes = np.mean(changes, axis=0)
        else:
            # 변화량이 없는 경우, 0으로 채워진 배열 반환
            mean_changes = np.zeros_like(keypoints_list[0])

        changes_list.append(mean_changes)

    return changes_list

In [9]:
def calculate_angle(point1, point2, point3):
    """
    세 점을 이용하여 두 벡터 사이의 각도를 계산합니다.
    :param point1, point2, point3: 각 점의 좌표를 나타내는 (x, y) 튜플이나 리스트.
    :return: 두 벡터 사이의 각도(도).
    """
    # 벡터 v1과 v2 생성
    v1 = np.array(point1) - np.array(point2)
    v2 = np.array(point3) - np.array(point2)

    # 벡터의 내적과 노름(크기)을 사용하여 각도(라디안) 계산
    angle_rad = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

    # 각도를 도로 변환
    angle_deg = np.degrees(angle_rad)

    return angle_deg

# 평균
def calculate_angle_changes(keypoints_data, point_indices):
    angle_changes_list = []
    for keypoints_list in keypoints_data['keypoints']:
        angles = []
        for frame_keypoints in keypoints_list:
            # 키포인트 데이터가 충분한지 확인
            if len(frame_keypoints) > max(point_indices):
                p1 = frame_keypoints[point_indices[0]][:2]  # x, y 좌표만 사용
                p2 = frame_keypoints[point_indices[1]][:2]
                p3 = frame_keypoints[point_indices[2]][:2]
                angle = calculate_angle(p1, p2, p3)
                angles.append(angle)
            else:
                # 충분한 데이터가 없는 경우 계산에서 제외
                continue

        if angles:  # 각도 데이터가 있을 경우에만 계산
            angle_changes = np.abs(np.diff(angles))
            mean_angle_change = np.mean(angle_changes)
            angle_changes_list.append(mean_angle_change)
        else:
            # 각도 데이터가 없는 경우 0으로 처리
            angle_changes_list.append(0)

    return np.array(angle_changes_list)

# 최솟값
def calculate_min_angle_changes(keypoints_data, point_indices):
    min_angle_changes_list = []
    for keypoints_list in keypoints_data['keypoints']:
        angles = []
        for frame_keypoints in keypoints_list:
            if len(frame_keypoints) > max(point_indices):
                p1 = frame_keypoints[point_indices[0]][:2]  # x, y 좌표만 사용
                p2 = frame_keypoints[point_indices[1]][:2]
                p3 = frame_keypoints[point_indices[2]][:2]
                angle = calculate_angle(p1, p2, p3)
                angles.append(angle)

        if angles:
            angle_changes = np.abs(np.diff(angles))
            min_angle_change = np.min(angle_changes) if len(angle_changes) > 0 else 0
            min_angle_changes_list.append(min_angle_change)
        else:
            min_angle_changes_list.append(0)

    return np.array(min_angle_changes_list)

# 최댓값
def calculate_max_angle_changes(keypoints_data, point_indices):
    max_angle_changes_list = []
    for keypoints_list in keypoints_data['keypoints']:
        angles = []
        for frame_keypoints in keypoints_list:
            if len(frame_keypoints) > max(point_indices):
                p1 = frame_keypoints[point_indices[0]][:2]  # x, y 좌표만 사용
                p2 = frame_keypoints[point_indices[1]][:2]
                p3 = frame_keypoints[point_indices[2]][:2]
                angle = calculate_angle(p1, p2, p3)
                angles.append(angle)

        if angles:
            angle_changes = np.abs(np.diff(angles))
            max_angle_change = np.max(angle_changes) if len(angle_changes) > 0 else 0
            max_angle_changes_list.append(max_angle_change)
        else:
            max_angle_changes_list.append(0)

    return np.array(max_angle_changes_list)


In [10]:
def calculate_enhanced_autocorrelation_features(keypoints_data):
    features_list = []  # 각 동영상의 향상된 자기상관성 특성을 저장할 리스트

    for keypoints_list in keypoints_data['keypoints']:
        changes = []
        prev_keypoints = None
        for keypoints in keypoints_list:
            keypoints = np.array(keypoints)
            if prev_keypoints is not None:
                change = np.linalg.norm(keypoints - prev_keypoints)
                changes.append(change)
            prev_keypoints = keypoints

        if changes:
            changes = np.array(changes)
            autocorrelation = np.correlate(changes - np.mean(changes), changes - np.mean(changes), mode='full')
            autocorrelation = autocorrelation[autocorrelation.size // 2:]  # 자기상관성 값 중 양의 지연만 고려

            # 향상된 특성 계산
            mean_autocorrelation = np.mean(autocorrelation)
            std_autocorrelation = np.std(autocorrelation)
            peak_count = np.sum(autocorrelation > (mean_autocorrelation + std_autocorrelation))  # 평균 이상의 피크 수

            features = [mean_autocorrelation, std_autocorrelation, peak_count]
        else:
            features = [0, 0, 0]

        features_list.append(features)

    return features_list

In [11]:
pkl_file_path = '/content/keypoints_data_augmented.pkl'

# pickle 파일 로드
with open(pkl_file_path, 'rb') as f:
     keypoints_data = pickle.load(f)

In [12]:
mean_keypoints_all_videos, labels = calculate_mean_keypoints_from_file(keypoints_data)
changes_list = calculate_keypoint_changes(keypoints_data)
autocorrelation_list = calculate_enhanced_autocorrelation_features(keypoints_data)

back_angle_changes_list1 = calculate_angle_changes(keypoints_data, (6,12,16))
back_angle_changes_list2 = calculate_angle_changes(keypoints_data, (5,11,15))
head_angle_changes_list1 = calculate_angle_changes(keypoints_data, (0,6,12))
head_angle_changes_list2 = calculate_angle_changes(keypoints_data, (0,5,11))
leg_angle_changes_list1 = calculate_angle_changes(keypoints_data, (12,14,16))
leg_angle_changes_list2 = calculate_angle_changes(keypoints_data, (11,13,15))
eye_angle_changes_list1 = calculate_angle_changes(keypoints_data, (1,5,9))
eye_angle_changes_list2 = calculate_angle_changes(keypoints_data, (2,6,10))
strech_angle_changes_list1 = calculate_angle_changes(keypoints_data, (5,8,10))
strech_angle_changes_list2 = calculate_angle_changes(keypoints_data, (6,7,9))
finger_angle_changes_list1 = calculate_angle_changes(keypoints_data, (0,8,10))
finger_angle_changes_list2 = calculate_angle_changes(keypoints_data, (0,7,9))

back_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (6,12,16))
back_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (5,11,15))
head_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (0,6,12))
head_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (0,5,11))
leg_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (12,14,16))
leg_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (11,13,15))
eye_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (1,5,9))
eye_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (2,6,10))
strech_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (5,8,10))
strech_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (6,7,9))
finger_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (0,8,10))
finger_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (0,7,9))

back_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (6,12,16))
back_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (5,11,15))
head_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (0,6,12))
head_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (0,5,11))
leg_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (12,14,16))
leg_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (11,13,15))
eye_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (1,5,9))
eye_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (2,6,10))
strech_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (5,8,10))
strech_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (6,7,9))
finger_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (0,8,10))
finger_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (0,7,9))

# 결과 확인 (예시로 첫 번째 동영상의 평균 키포인트 출력)
# print(angle_changes_list2[0])

In [13]:
features = []
mean_keypoints_all_videos = np.array(mean_keypoints_all_videos)
changes_list = np.array(changes_list)
autocorrelation_list = np.array(autocorrelation_list)

for i in range(len(mean_keypoints_all_videos)):
    combined_feature = np.concatenate([mean_keypoints_all_videos[i].flatten(), changes_list[i].flatten(), autocorrelation_list[i].flatten(),
    # fft_features_list[i].flatten(),
    [back_max_angle_changes_list1[i] - back_min_angle_changes_list1[i],
    back_max_angle_changes_list2[i] - back_min_angle_changes_list2[i],
    head_max_angle_changes_list1[i] - head_min_angle_changes_list1[i],
    head_max_angle_changes_list2[i] - head_min_angle_changes_list2[i],
    leg_max_angle_changes_list1[i] - leg_min_angle_changes_list1[i],
    leg_max_angle_changes_list2[i] - leg_min_angle_changes_list2[i],
    eye_max_angle_changes_list1[i] - eye_min_angle_changes_list1[i],
    eye_max_angle_changes_list2[i] - eye_min_angle_changes_list2[i],
    strech_max_angle_changes_list1[i] - strech_min_angle_changes_list1[i],
    strech_max_angle_changes_list2[i] - strech_min_angle_changes_list2[i],
    finger_max_angle_changes_list1[i] - finger_min_angle_changes_list1[i],
    finger_max_angle_changes_list2[i] - finger_min_angle_changes_list2[i]]])

    features.append(combined_feature)


In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import optuna

# features와 labels를 넘파이 배열로 변환
features = np.array(features)
labels = np.array(labels)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def objective(trial):
    # 모델 생성 함수
    def create_model(trial):
        n_layers = trial.suggest_int('n_layers', 1, 3)
        model = Sequential()
        for i in range(n_layers):
            num_hidden = trial.suggest_int(f'n_units_l{i}', 4, 128, log=True)
            model.add(Dense(num_hidden, activation='relu'))
            dropout_rate = trial.suggest_float(f'dropout_l{i}', 0.0, 0.5)
            model.add(Dropout(rate=dropout_rate))
        model.add(Dense(6, activation='softmax'))  # 클래스 수에 따라 조정
        return model

    # 모델 컴파일
    model = create_model(trial)
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # 모델 훈련
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=0)

    # 검증 정확도의 최댓값 반환
    val_accuracy = max(history.history['val_accuracy'])
    return val_accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # 시도할 횟수 조정

print(f'Best trial: {study.best_trial.params}')

model = Sequential([
    Dense(study.best_trial.params['n_units_l0'], activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(study.best_trial.params['dropout_l0']),
    Dense(6, activation='softmax')  # 클래스 수에 따라 조정
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=study.best_trial.params['lr']),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 최적의 모델 훈련
model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=50, batch_size=32)

# 모델을 SavedModel 형식으로 저장
SAVED_MODEL_PATH = "saved_model"
model.save(SAVED_MODEL_PATH)

# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
tflite_model = converter.convert()

# 변환된 모델 저장
TFLITE_MODEL_PATH = "converted_model.tflite"
with open(TFLITE_MODEL_PATH, "wb") as f:
    f.write(tflite_model)

[I 2024-04-10 09:43:46,681] A new study created in memory with name: no-name-79594465-a110-4eb8-9246-65760ae1f3d2
[I 2024-04-10 09:44:08,774] Trial 0 finished with value: 0.22413793206214905 and parameters: {'n_layers': 2, 'n_units_l0': 4, 'dropout_l0': 0.16367793401299213, 'n_units_l1': 108, 'dropout_l1': 0.05209399229679823, 'lr': 3.922651545461333e-05}. Best is trial 0 with value: 0.22413793206214905.
[I 2024-04-10 09:44:14,474] Trial 1 finished with value: 0.16379310190677643 and parameters: {'n_layers': 2, 'n_units_l0': 14, 'dropout_l0': 0.4729991141946351, 'n_units_l1': 9, 'dropout_l1': 0.4851322620572945, 'lr': 8.405270003214766e-05}. Best is trial 0 with value: 0.22413793206214905.
[I 2024-04-10 09:44:19,635] Trial 2 finished with value: 0.37931033968925476 and parameters: {'n_layers': 2, 'n_units_l0': 5, 'dropout_l0': 0.1922455262735306, 'n_units_l1': 8, 'dropout_l1': 0.1457007075419406, 'lr': 0.011358717857603005}. Best is trial 2 with value: 0.37931033968925476.
[I 2024-04-1

Best trial: {'n_layers': 1, 'n_units_l0': 75, 'dropout_l0': 0.011089658358364285, 'lr': 0.027035056291659778}
Epoch 1/50
19/19 [==============================] - 1s 13ms/step - loss: 1.2268 - accuracy: 0.6293 - val_loss: 1.2072 - val_accuracy: 0.6828
Epoch 2/50
19/19 [==============================] - 0s 5ms/step - loss: 0.5462 - accuracy: 0.8328 - val_loss: 0.5785 - val_accuracy: 0.8207
Epoch 3/50
19/19 [==============================] - 0s 5ms/step - loss: 0.2834 - accuracy: 0.8983 - val_loss: 0.4713 - val_accuracy: 0.8897
Epoch 4/50
19/19 [==============================] - 0s 5ms/step - loss: 0.1947 - accuracy: 0.9397 - val_loss: 0.5944 - val_accuracy: 0.8552
Epoch 5/50
19/19 [==============================] - 0s 5ms/step - loss: 0.1458 - accuracy: 0.9466 - val_loss: 1.1314 - val_accuracy: 0.8276
Epoch 6/50
19/19 [==============================] - 0s 5ms/step - loss: 0.5147 - accuracy: 0.8810 - val_loss: 1.1153 - val_accuracy: 0.8069
Epoch 7/50
19/19 [==============================]